In [ ]:
import os
import pandas as pd

# Gộp các file đầu vào thành 1 Dataframe

In [ ]:
def load_documents(num_files):
    all_documents = []

    for sample_num in range(1, num_files + 1):
        train_file_name = f"/kaggle/working/train_ner_{sample_num}.csv"
        test_file_name = f"/kaggle/working/test_ner_{sample_num}.csv"

        try:
            train_df = pd.read_csv(train_file_name, sep=",", encoding="utf-8-sig")
            all_documents.append(train_df)
        except FileNotFoundError:
            print(f"Train file {train_file_name} not found.")

        try:
            test_df = pd.read_csv(test_file_name, sep=",", encoding="utf-8-sig")
            all_documents.append(test_df)
        except FileNotFoundError:
            print(f"Test file {test_file_name} not found.")

    combined_df = pd.concat(all_documents, ignore_index=True) if all_documents else pd.DataFrame()

    return combined_df


In [ ]:
df = load_documents(3)

# Xây dựng chủ đề ẩn bằng LDA

In [ ]:
from gensim import corpora
from gensim.models import LdaModel

!pip3 install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

## Xây dựng Bags-of-words

In [ ]:
import re

def preprocess(text):
    
    keep_pattern = r'(?<=[a-zA-Z0-9])([.,])(?=[a-zA-Z0-9])|(?<=[a-zA-Z])\.(?=\s|,|$)'
    remove_pattern = r'[.,](?=\s|$|[^a-zA-Z0-9])'

    
    text = re.sub(keep_pattern, r'\1', text)

    
    text = re.sub(remove_pattern, ' ', text)

    
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'[/-]', ' ', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\s{2,}', ' ', text)

    
    return text.strip().split()


df['content'] = df['content'].astype(str)
documents = df['content'].map(preprocess).tolist()


dictionary = corpora.Dictionary(documents)


corpus = [dictionary.doc2bow(doc) for doc in documents]

## Xây dựng LDA mức 1

### Đánh giá số lượng cụm 

In [ ]:
from gensim.models import CoherenceModel


def compute_perplexity(model, corpus_data):
    return model.log_perplexity(corpus_data)


def compute_coherence(model, texts_data, word_dict, coherence_method='c_v'):
    coherence = CoherenceModel(
        model=model,
        texts=texts_data,
        dictionary=word_dict,
        coherence=coherence_method
    )
    return coherence.get_coherence()


In [ ]:
import matplotlib.pyplot as plt
from gensim.models import LdaModel


num_topics_list = [9, 10, 11, 12]  
results = []


for num_topics in num_topics_list:
   
    model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
    
    
    perplexity = compute_perplexity(lda_model, corpus)
    coherence = compute_coherence(lda_model, documents, dictionary, coherence_type='c_v')
    
   
    results.append({
        "num_topics": num_topics,
        "perplexity": perplexity,
        "coherence_score": coherence
    })

In [ ]:

num_topics_list = [result["num_topics"] for result in results]
perplexity_list = [result["perplexity"] for result in results]
coherence_list = [result["coherence_score"] for result in results]


fig, ax1 = plt.subplots()


ax1.set_xlabel('Number of Topics')
ax1.set_ylabel('Perplexity', color='tab:blue')
ax1.plot(num_topics_list, perplexity_list, color='tab:blue', marker='o', label='Perplexity')
ax1.tick_params(axis='y', labelcolor='tab:blue')


ax2 = ax1.twinx()  
ax2.set_ylabel('Coherence Score', color='tab:green')
ax2.plot(num_topics_list, coherence_list, color='tab:green', marker='o', label='Coherence')
ax2.tick_params(axis='y', labelcolor='tab:green')


plt.title('Perplexity and Coherence vs. Number of Topics')
fig.tight_layout()  
plt.show()

### Xây dựng thống kê cho các cụm 

In [ ]:
num_topics = 12
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

In [ ]:
from collections import Counter

def get_stat(lda_model, corpus):
    stat = []
    for doc_bow in corpus:
        topic_probs = lda_model.get_document_topics(doc_bow)
        stat_count = max(topic_probs, key=lambda x: x[1])[0]
        stat.append(stat_count)
    return stat

stat = get_stat(lda_model, corpus)

topic_counts = Counter(stat)

print("\n Thống kê các cụm chủ đề:")
for topic, count in topic_counts.items():
    print(f"Chủ đề {topic}: {count} tài liệu")

print(f"\nSố tài liệu: {sum(topic_counts.values())}")

### Lấy từ khóa của mỗi chủ đề

In [ ]:

def print_topic_words(model, num_topics):
    # Lặp qua từng chủ đề và lấy từ khóa
    for idx in range(num_topics):
        topic_terms = model.show_topic(idx, topn=200)
        topic_keywords = [term for term, _ in topic_terms]
        print(f"Chủ đề {idx}: {', '.join(topic_keywords)}")


print_topic_words(lda_model, num_topics)

### Phân phối chủ đề cho các tài liệu

In [ ]:
import pandas as pd

def get_topic_distribution_for_documents(lda_model, corpus, content_list, top_n=5):
  
    result = []


    for idx, doc_bow in enumerate(corpus):
       
        topic_probabilities = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        
      
        sorted_topics = sorted(topic_probabilities, key=lambda x: x[1], reverse=True)[:top_n]

        
        topic_info = {"content": content_list[idx]}
        for rank, (topic_id, prob) in enumerate(sorted_topics):
            topic_info[f"Chủ đề {rank + 1}"] = f"Chủ đề {topic_id} ({prob * 100:.2f}%)"
        
       
        result.append(topic_info)
    
    
    return pd.DataFrame(result)


content_list = df['content'].tolist()


top_n_topics = 3


topic_distribution_df = get_topic_distribution_for_documents(
    lda_model, corpus, content_list, top_n=top_n_topics
)


output_file = "/kaggle/working/phân_phối_của_chủ dề.csv"
topic_distribution_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"File đã được lưu tại: {output_file}")


# Diễn giải các chủ đề

In [ ]:
import pandas as pd

file_name = "/kaggle/working/phân_phối_của_chủ dề.csv"
df = pd.read_csv(file_name, encoding="utf-8-sig")

columns_to_drop = df.columns[4:]
df.drop(columns=columns_to_drop, inplace=True)
print(df)

In [ ]:
import pandas as pd

output_file_name = "/kaggle/working/diễn_giải.csv"

labels_map = {
    "Chủ đề 0": "Thể thao",
    "Chủ đề 1": "Thương mại",
    "Chủ đề 2": "Đô thị hóa",
    "Chủ đề 3": "Thời tiết",
    "Chủ đề 4": "Luật pháp",
    "Chủ đề 5": "Đời sống",
    "Chủ đề 6": "Thế giới",
    "Chủ đề 7": "Kinh tế xã hội",
    "Chủ đề 8": "Chính trị",
    "Chủ đề 9": "Giáo dục",
    "Chủ đề 10": "Sức khỏe",
    "Chủ đề 11": "Xã hội",
}

df['Chủ đề 1'] = df['Chủ đề 1'].str.extract(r"(Chủ đề \d+)")[0]
df['Chủ đề 1'] = df['Chủ đề 1'].map(labels_map)
df.rename(columns={"Chủ đề 1": "Chủ đề chính"}, inplace=True)

df.to_csv(output_file_name, index=False, encoding="utf-8-sig")

In [ ]:
output_folder = "/kaggle/working/diễn_giải"
df_grouped = df.groupby("Chủ đề chính")

print(df_grouped)

file_paths = {}
for group_name, group_data in df_grouped:
    file_name = f"{output_folder}/{group_name.replace(' ', '_')}.csv"
    group_data.to_csv(file_name, index=False, encoding="utf-8-sig")
    file_paths[group_name] = file_name

print("Các file đã được lưu:", file_paths)